In [4]:
import cv2
import imutils

In [ ]:
import cv2
import imutils

def detect_people_video(file_path, output_file):
    # Initialize the HOG person detector
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    # Open the video file
    cap = cv2.VideoCapture(file_path)

    # Get the frame width and height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    result = cv2.VideoWriter('filename.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, (frame_width, frame_height)) 

    while cap.isOpened():
        # Capture each frame of the video
        ret, frame = cap.read()
        if ret == True:
            high_confidence, moderate_confidence, low_confidence = 0, 0, 0
            
            # Resize the frame for faster processing
            resized_frame = imutils.resize(frame, width=min(1000, frame.shape[1]))
            
            # Convert the resized frame to grayscale
            img_gray = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
            
            # Detect people in the resized grayscale frame
            rects, weights = hog.detectMultiScale(img_gray, padding=(4, 4), scale=1.02)
            
            # Draw rectangles based on confidence levels
            for i, (x, y, w, h) in enumerate(rects):
                if weights[i] < 0.13:
                    continue
                elif weights[i] < 0.3 and weights[i] > 0.13:
                    cv2.rectangle(resized_frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                    low_confidence += 1
                if weights[i] < 0.7 and weights[i] > 0.3:
                    cv2.rectangle(resized_frame, (x, y), (x+w, y+h), (50, 122, 255), 2)
                    moderate_confidence += 1
                if weights[i] > 0.7:
                    cv2.rectangle(resized_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    high_confidence += 1
            
            # Write confidence labels on the frame
            cv2.putText(resized_frame, 'High Confidence: ' + str(high_confidence), (10, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(resized_frame, 'Moderate Confidence: ' + str(moderate_confidence), (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50, 122, 255), 2)
            cv2.putText(resized_frame, 'Low Confidence: ' + str(low_confidence), (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Write frame to output video
            result.write(resized_frame)

        else:
            break

    # Release the video capture object and release the output video
    cap.release()
    result.release()

# Example usage:
file_path = 'Downloads/Testing_video.mp4'
output_file = 'output.avi'
detect_people_video(file_path, output_file)

In [2]:
# Initializing the HOG person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cap = cv2.VideoCapture('Downloads/Testing_video.mp4')

# Counter variable to track the number of red boxes
red_box_count = 0

while cap.isOpened():
    # Reading the video stream
    ret, image = cap.read()
    if ret:
        image = imutils.resize(image, width=min(500, image.shape[1]))

        # Detecting all the regions in the Image that has pedestrians inside it
        (regions, _) = hog.detectMultiScale(image,
                                            winStride=(4, 4),
                                            padding=(1, 1),
                                            scale=0.5)

        # Drawing the regions in the Image
        red_box_count = 0
        for (x, y, w, h) in regions:
            cv2.rectangle(image, (x, y),
                          (x + w, y + h),
                          (0, 0, 255), 2)
            red_box_count += 1

        # Displaying the count in the top right corner
        cv2.putText(image, f'Humans: {red_box_count}', (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Showing the output Image
        cv2.imshow("Image", image)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()